In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm
# from tqdm.notebook import tqdm

from collections import OrderedDict, defaultdict
import itertools
from functools import partial
import datetime
from joblib import Parallel, delayed
import copy
import json

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data, get_district_time_series

from models.seir.seir_testing import SEIR_Testing
from main.seir.optimiser import Optimiser
from utils.loss import Loss_Calculator
from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv, plot_forecast

## Load Covid19india Data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Pune'), 
                     ('Maharashtra', 'Mumbai'), 
                     ('Rajasthan', 'Jaipur'), 
                     ('Gujarat', 'Ahmedabad'), 
                     ('Karnataka', 'Bengaluru Urban'),
                     ('Delhi', None)]

districts_to_show = [('Maharashtra', 'Pune')]

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(dataframes, state, district, train_period=7, val_period=7, 
                                                                     data_from_tracker=False, filename="../../data/data/official-pune-21-05-20.csv", initialisation='intermediate', num_evals=2000,
                                                                     which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(dataframes, state, district, train_period=7, val_period=0, num_evals=2000,
                                                                     train_on_val=True, data_from_tracker=False, filename="../../data/data/official-pune-21-05-20.csv",initialisation='intermediate',
                                                                     which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(dataframes, state, district, train_period=7, val_period=7, num_evals=1000,
                                                                     data_from_tracker=False, filename='../../data/data/official-mumbai.csv', 
                                                                     initialisation='intermediate',
                                                                     which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(dataframes, state, district, train_period=7, val_period=0, num_evals=1000,
                                                                     data_from_tracker=False, filename='../../data/data/official-mumbai.csv',
                                                                     initialisation='intermediate', train_on_val=True, 
                                                                     which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])

## Create Master Loss Dataframe

### M1 Loss

In [ ]:
starting_key = list(predictions_dict.keys())[0]

loss_columns = pd.MultiIndex.from_product([predictions_dict[starting_key]['m1']['df_loss'].columns, predictions_dict[starting_key]['m1']['df_loss'].index])
loss_index = predictions_dict.keys()

df_loss_master = pd.DataFrame(columns=loss_columns, index=loss_index)
for key in predictions_dict.keys():
    df_loss_master.loc[key, :] = np.around(predictions_dict[key]['m1']['df_loss'].values.T.flatten().astype('float'), decimals=2)
    
df_loss_master

### M2 Loss

In [ ]:
starting_key = list(predictions_dict.keys())[0]

loss_columns = pd.MultiIndex.from_product([predictions_dict[starting_key]['m2']['df_loss'].columns, predictions_dict[starting_key]['m2']['df_loss'].index])
loss_index = predictions_dict.keys()

df_loss_master = pd.DataFrame(columns=loss_columns, index=loss_index)
for key in predictions_dict.keys():
    df_loss_master.loc[key, :] = np.around(predictions_dict[key]['m2']['df_loss'].values.T.flatten().astype('float'), decimals=2)
    
df_loss_master

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    plot_forecast(predictions_dict[region], region, both_forecasts=False, error_bars=True)

In [ ]:
for region in predictions_dict.keys():
    plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, initialisation='intermediate', train_period=7, icu_fraction=0.02)

In [ ]:
write_csv(df_output, '../../output-{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

In [ ]:
starting_key=list(predictions_dict.keys())[0]

In [ ]:
predictions_dict[starting_key]

## Create Multiple Forecasts For Top 10 Param Sets

In [ ]:
losses_array = np.array([trial['result']['loss'] for trial in predictions_dict[starting_key]['m2']['trials']])
least_losses_indices = np.argsort(losses_array)
np.sort(losses_array)

In [ ]:
np.sort(losses_array)[1:100]

In [ ]:
params_array = []
for trial in predictions_dict[starting_key]['m2']['trials']:
    params_dict = copy.copy(trial['misc']['vals'])
    for key in params_dict.keys():
        params_dict[key] = params_dict[key][0]
    params_array.append(params_dict)

params_array = np.array(params_array)

In [ ]:
np.sort(losses_array)[:10]

In [ ]:
predictions_array = [get_forecast(predictions_dict[starting_key], best_params=params_dict) for params_dict in params_array[least_losses_indices[:10]]]

In [ ]:
params_dict = params_array[least_losses_indices[:10]]
params_dict

In [ ]:
df_true = predictions_dict[starting_key]['m2']['df_district']
color_idx=np.linspace(0,1,10)
fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(df_true['date'], df_true['hospitalised'],
        '-o', color='orange', label='Active Cases (Observed)')
for i, df_prediction in enumerate(predictions_array):
    loss_value = np.around(np.sort(losses_array)[:10][i], 3)
    params_dict = params_array[least_losses_indices[:10]]
    true_r0 = np.around(params_dict[i]['lockdown_R0'], 3)
    if true_r0 > 1.7:
        continue
    sns.lineplot(x="date", y="hospitalised", data=df_prediction,
                 ls='-', color=plt.cm.summer(color_idx[i]), label='Active Cases ({})'.format(loss_value))
    plt.text(x=df_prediction['date'].iloc[-1], y=df_prediction['hospitalised'].iloc[-1], s="{0:.3f}".format(true_r0))
    
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.ylabel('No of People', fontsize=16)
# plt.yscale('log')
plt.xlabel('Time', fontsize=16)
plt.legend()
plt.title('Forecast - ({} {})'.format(region[0], region[1]), fontsize=16)
plt.grid(True)
plt.show()  